In [1]:
# In this scripts, you will laern how to do calibartion and zero-shot learning
# We use manual verbalizer and knowledgeable verbalizer as examples.
from tqdm import tqdm
from openprompt.data_utils.text_classification_dataset import ImdbProcessor
import torch
from openprompt.data_utils.utils import InputExample
from openprompt.trainer import ClassificationRunner

dataset = {}
dataset['train'] = ImdbProcessor().get_train_examples("../datasets/TextClassification/imdb/")
# dataset['validation'] = ImdbProcessor().get_dev_examples("../datasets/TextClassification/imdb/")
dataset['test'] = ImdbProcessor().get_test_examples("../datasets/TextClassification/imdb/")
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-base")

In [4]:
# just a sanity check of what these data look like in first place
# import pandas as pd
# raw_data = pd.read_csv("../datasets/TextClassification/SST-2/train.tsv", sep='\t')

In [6]:
# raw_data

In [3]:
# plm

In [4]:
# !pip install yacs
# !pip install tensorboardX
# !pip install rouge==1.0.0

In [9]:
dataset['train']

[{
   "guid": "0",
   "label": 1,
   "meta": {},
   "text_a": "Lorna Green ( Janine Reynaud ) is a performance artist for wealthy intellectuals at a local club. She falls prey to her fantasies as the promise of romantic interludes turn into murder as she kills those who believe that sex is on the horizon. It's quite possible that, through a form of hypnotic suggestion, someone ( ..a possible task master pulling her strings like a puppet ) is guiding Lorna into killing those she comes across in secluded places just when it appears that love-making is about to begin. After the murders within her fantasies are committed, Lorna awakens bewildered, often clueless as to if what she was privy to within her dreams ever took place in reality. If someone asked me how to describe this particular work from Franco, I'd say it's elegant & difficult. By now, you've probably read other user comments befuddled by what this film is about, since a large portion of it takes place within the surreal atmosp

In [2]:

from openprompt.prompts import ManualTemplate, SoftTemplate
mytemplate = ManualTemplate(tokenizer=tokenizer).from_file("../scripts/TextClassification/imdb/manual_template.txt", choice=0)
from openprompt import PromptDataLoader



# ## Define the verbalizer
from openprompt.prompts import ManualVerbalizer, KnowledgeableVerbalizer

# this is a user provided set of label words?
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=2).from_file("../scripts/TextClassification/imdb/knowledgeable_verbalizer.txt")

In [12]:
len(myverbalizer.label_words)

2

In [15]:
# myverbalizer.label_words[1]

In [9]:
mytemplate.text

[{'add_prefix_space': '', 'text': 'It was'},
 {'add_prefix_space': ' ', 'mask': None},
 {'add_prefix_space': '', 'text': '.'},
 {'add_prefix_space': ' ', 'placeholder': 'text_a'}]

In [ ]:
mytemplat

In [7]:
# load promptforclassification model class

from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = mytemplate,
    plm = plm,
    verbalizer = myverbalizer,
)

In [11]:
promptModel

PromptForClassification(
  (prompt_model): PromptModel(
    (plm): RobertaForMaskedLM(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0): RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=1024, out_features=1024, bias=True)
                  (key): Linear(in_features=1024, out_features=1024, bias=True)
                  (value): Linear(in_features=1024, out_features=1024, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
   

In [8]:
# prompt data loader

from openprompt import PromptDataLoader
train_dataloader = PromptDataLoader(
    dataset = dataset['train'],
    tokenizer = tokenizer, 
    template = mytemplate, 
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=512, decoder_max_length=3,truncate_method="tail",
    batch_size = 4
)


test_dataloader = PromptDataLoader(
    dataset = dataset['test'],
    tokenizer = tokenizer, 
    template = mytemplate, 
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=512, decoder_max_length=3,truncate_method="tail",
    batch_size = 4
)

tokenizing: 25000it [00:38, 641.12it/s]
tokenizing: 25000it [00:37, 666.10it/s]


In [23]:
train_dataloader.wrapped_dataset

[[[{'text': 'It was', 'loss_ids': 0, 'shortenable_ids': 0},
   {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0},
   {'text': '.', 'loss_ids': 0, 'shortenable_ids': 0},
   {'text': ' Lorna Green ( Janine Reynaud ) is a performance artist for wealthy intellectuals at a local club. She falls prey to her fantasies as the promise of romantic interludes turn into murder as she kills those who believe that sex is on the horizon. It\'s quite possible that, through a form of hypnotic suggestion, someone ( ..a possible task master pulling her strings like a puppet ) is guiding Lorna into killing those she comes across in secluded places just when it appears that love-making is about to begin. After the murders within her fantasies are committed, Lorna awakens bewildered, often clueless as to if what she was privy to within her dreams ever took place in reality. If someone asked me how to describe this particular work from Franco, I\'d say it\'s elegant & difficult. By now, you\'ve probabl

In [9]:
# important - set model to cuda
use_cuda = True
if use_cuda:
    promptModel = promptModel.cuda()

In [10]:
# learning rate and optimizer
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in promptModel.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in promptModel.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

In [16]:
for step, inputs in enumerate(train_dataloader):
    print(inputs)
    break

{"input_ids": [[0, 243, 21, 50264, 4, 226, 4244, 102, 1628, 36, 1133, 833, 31268, 5247, 4839, 16, 10, 819, 3025, 13, 8581, 37306, 23, 10, 400, 950, 4, 264, 5712, 18644, 7, 69, 35591, 25, 5, 4198, 9, 8728, 3222, 462, 21683, 1004, 88, 1900, 25, 79, 10469, 167, 54, 679, 14, 2099, 16, 15, 5, 13290, 4, 85, 18, 1341, 678, 14, 6, 149, 10, 1026, 9, 39040, 636, 10791, 6, 951, 36, 29942, 102, 678, 3685, 4710, 6539, 69, 22052, 101, 10, 29771, 4839, 16, 17769, 226, 4244, 102, 88, 2429, 167, 79, 606, 420, 11, 842, 26441, 2127, 95, 77, 24, 2092, 14, 657, 12, 5349, 16, 59, 7, 1642, 4, 572, 5, 13119, 624, 69, 35591, 32, 2021, 6, 226, 4244, 102, 19267, 45830, 33304, 3215, 6, 747, 36776, 25, 7, 114, 99, 79, 21, 20955, 219, 7, 624, 69, 7416, 655, 362, 317, 11, 2015, 4, 318, 951, 553, 162, 141, 7, 6190, 42, 1989, 173, 31, 14185, 6, 38, 1017, 224, 24, 18, 14878, 359, 1202, 4, 870, 122, 6, 47, 348, 1153, 1166, 97, 3018, 1450, 28, 506, 23521, 30, 99, 42, 822, 16, 59, 6, 187, 10, 739, 4745, 9, 24, 1239, 317, 

In [ ]:
# # run training
# for epoch in range(1):
#     tot_loss = 0 
#     for step, inputs in enumerate(tqdm(train_dataloader)):
#         if use_cuda:
#             inputs = inputs.cuda()
#         logits = promptModel(inputs)
#         labels = inputs['label']
#         loss = loss_func(logits, labels)
#         loss.backward()
#         tot_loss += loss.item()
#         optimizer.step()
#         optimizer.zero_grad()
#         if step %100 ==1:
#             print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

In [11]:
def fit(model, train_dataloader, val_dataloader, loss_func, optimizer, epochs = 2, cuda = True):
    best_score = 0.0
    for epoch in range(epochs):
        train_epoch(model, train_dataloader, loss_func, optimizer, epoch, cuda = True)
        score = evaluate(model, val_dataloader)
        if score > best_score:
            best_score = score
    return best_score
        

def train_epoch(model, train_dataloader, loss_func, optimizer, epoch, cuda = True):
    model.train()
    tot_loss = 0
    pbar = tqdm(test_dataloader)
    for step, inputs in enumerate(pbar):
        if cuda:            
            inputs = inputs.cuda()
        logits = model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        tot_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if step %500 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

def evaluate(model, val_dataloader, cuda = True):
    model.eval()
    allpreds = []
    alllabels = []
    with torch.no_grad():
        for step, inputs in enumerate(val_dataloader):
            if cuda:
                
                inputs = inputs.cuda()
            logits = model(inputs)
            labels = inputs['label']
            alllabels.extend(labels.cpu().tolist())
            allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
    print(f"accuracy was: {acc}")
    return acc

In [12]:
# run simple training and eval - will updated with more metrics


score = fit(promptModel, train_dataloader, test_dataloader, loss_func, optimizer, cuda = True)


test_score = evaluate(promptModel, test_dataloader)
print(f"Test score was: {test_score}")

  0%|          | 1/6250 [00:00<34:29,  3.02it/s]

Epoch 0, average loss: 0.08470617861530627


  8%|▊         | 501/6250 [02:27<28:12,  3.40it/s]

Epoch 0, average loss: 0.0003376139718358617


 16%|█▌        | 1001/6250 [04:55<25:56,  3.37it/s]

...

Epoch 1, average loss: 0.00931973084256157


 96%|█████████▌| 6001/6250 [29:52<01:14,  3.34it/s]

Epoch 1, average loss: 0.008543511870613286


100%|██████████| 6250/6250 [31:07<00:00,  3.35it/s]


Test score was: 0.5


# Using soft template 

In [13]:
# with soft template

softTemplate = SoftTemplate(model=plm, tokenizer=tokenizer, text='{"placeholder":"text_a"} {"special": "</s>"} {"mask"}',num_tokens=100)


In [26]:
softTemplate

SoftTemplate(
  (raw_embedding): Embedding(50265, 768, padding_idx=1)
)

In [14]:

softpromptModel = PromptForClassification(
    template = softTemplate,
    plm = plm,
    verbalizer = myverbalizer,
)

In [15]:
# from openprompt import PromptDataLoader
soft_train_dataloader = PromptDataLoader(
    dataset = dataset['train'],
    tokenizer = tokenizer, 
    template = softTemplate, 
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=512, decoder_max_length=3,truncate_method="tail"
)


soft_test_dataloader = PromptDataLoader(
    dataset = dataset['test'],
    tokenizer = tokenizer, 
    template = softTemplate, 
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=512, decoder_max_length=3,truncate_method="tail"
)


tokenizing: 25000it [00:34, 721.65it/s]
tokenizing: 25000it [00:33, 741.78it/s]


In [16]:
# important - set model to cuda
use_cuda = True
if use_cuda:
    softpromptModel = softpromptModel.cuda()

In [17]:
# learning rate and optimizer
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in softpromptModel.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in softpromptModel.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

In [ ]:
# run simple training and eval - will updated with more metrics


soft_score = fit(softpromptModel, soft_train_dataloader, soft_test_dataloader, loss_func, optimizer, cuda = True)


soft_test_score = evaluate(softpromptModel, soft_test_dataloader)
print(f"Test score was: {test_score}")